# Preliminaries

In [1]:
# import modules
import numpy as np

In [2]:
# base classes

class Node:
    pass

class Tree:
    def __init__(self):
        self.root = Node()
    
    def find_leaf(self, x):
        node = self.root
        while hasattr(node, "feature"):
            j = node.feature
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node

# Density Tree

In [3]:
class DensityTree(Tree):
    def __init__(self):
        super(DensityTree, self).__init__()
        
    def train(self, data, prior):
        '''
        data: the feature matrix for the digit under consideration
        prior: the prior probability of this digit
        '''
        self.prior = prior
        N, D = data.shape
        D_try = int(np.sqrt(D)) # number of features to consider for each split decision

        # filter features and initialize bounding box
        # (If m[j] == M[j] for some j, the bounding box has zero volume, 
        #  causing divide-by-zero later on. We must ignore these features
        #  and adjust the bounding box accordingly.)
        m, M = np.min(data, axis=0), np.max(data, axis=0)
        valid_features   = np.where(m != M)[0]
        invalid_features = np.where(m == M)[0]
        M[invalid_features] = m[invalid_features] + 1

        # initialize the root node
        self.root.data = data
        self.root.box = m.copy(), M.copy()
        stack = [self.root]

        n_min = 20 # termination criterion: don't split if node contains fewer instances
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min:
                
                # Call 'make_density_split_node()' with 'D_try' randomly selected 
                # indices from 'valid_features'. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                #choose randomly D_try elements from valid_features
                feature_indices=valid_features[np.random.permutation(len(valid_features))[:D_try]]
                #put left and right to end of stack since pop() gets the last item
                stack.extend(make_density_split_node(node,N,feature_indices)) 
                
            else:
                # Call 'make_density_leaf_node()' to turn 'node' into a leaf node.
                node=make_density_leaf_node(node,N)

    def predict(self, x):
        leaf = self.find_leaf(x)        
        # compute p(x | y) * p(y)
        return leaf.response*self.prior

In [4]:
def volume(plus,minus):
    '''
    minus: numpy array of all x^-
    plus:       -''-          x^+
    '''
    #return volume
    return np.prod(plus-minus)

In [5]:
def score(N, m ,M , x_lambda, x_rho, S_lambda, S_rho, j):
    m_lambda, m_rho, M_lambda, M_rho = m.copy(), m.copy(), M.copy(), M.copy()
    m_lambda[j], m_rho[j], M_lambda[j], M_rho[j] = x_lambda[0], x_rho[0], x_lambda[1], x_rho[1]
    V_lambda = volume(M_lambda, m_lambda)
    V_rho = volume(M_rho, m_rho)
    looErr = len(S_lambda)/(N * V_lambda)*(len(S_lambda)/N - 2 * (len(S_lambda) - 1) / (N - 1)) +\
             len(S_rho)   /(N * V_rho)*   (len(S_rho)   /N - 2 * (len(S_rho) - 1)    / (N - 1))
    return looErr
  
def make_density_split_node(node, N, feature_indices):
    '''
    node: the node to be split
    N:    the total number of training instances for the current class
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape
    m, M = node.box

    # find best feature j (among 'feature_indices') and best threshold t for the split
    # Hint: For each feature considered, first remove duplicate feature values using 
    # 'np.unique()'. Describe here why this is necessary.
    best_splits = []
    best_scores = []
    for j in feature_indices:      
            sorted_data = np.sort(node.data.T[j])
            # same argumentation
            # filter features and initialize bounding box
            # (If m[j] == M[j] for some j, the bounding box has zero volume, 
            #  causing divide-by-zero later on. We must ignore these features
            sorted_data = np.unique(sorted_data)
            x_j = np.insert(sorted_data, 0, 0)
            x_j_plus = np.insert(sorted_data, len(sorted_data), 0)
            t_j = ((x_j + x_j_plus)/2)[1:-2]
            scores = []
            for t in t_j:
                S_lambda = sorted_data[sorted_data <= t]
                x_lambda = S_lambda[[0,-1]]
                S_rho = sorted_data[sorted_data > t]
                x_rho = S_rho[[0,-1]]
                scores.append(score(N, m ,M , x_lambda, x_rho, S_lambda, S_rho, j))
            if scores==[]:
                continue
            best_splits.append(t_j[np.argmin(scores)])
            best_scores.append(min(scores))
    
    index=np.argmin(best_scores)
    split_feature = feature_indices[index]
    split_threshold = best_scores[index]
    threshold = best_splits[index]
    
    # create children, data yet to be added, only definition of boxes
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and bounding boxes
    # according to the optimal split found above
    # split data into left.data and right.data
    subData=node.data[:,split_feature] #vllt index?
    indices=np.argsort(subData)
    splitIndex=np.argwhere(subData[indices]>=split_threshold)[0][0]
    
    # data smaller than threshold into left 
    left.data=node.data[indices,:][:splitIndex,:]
    right.data =node.data[indices,:][splitIndex:,:]
    
    # set left.box and right.box:
    m_rho, M_lambda = m.copy(), M.copy()
    m_rho[split_feature], M_lambda[split_feature] = threshold, threshold
    left.box = (m, M_lambda)
    right.box = (m_rho, M)
    
    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left,node.right=left,right
    node.feature=split_feature

    # return the children (to be placed on the stack)
    return left, right

In [6]:
def make_density_leaf_node(node, N):
    '''
    node: the node to become a leaf
    N:    the total number of training instances for the current class
    '''
    # compute and store leaf response
    # compute volume Vl
    Vl=volume(*node.box)
    # response
    response = node.data.shape[0]/(N*Vl)
    node=Node()
    node.response=response
    return node

# Decision Tree

In [7]:
class DecisionTree(Tree):
    def __init__(self):
        super(DecisionTree, self).__init__()
        
    def train(self, data, labels):
        '''
        data: the feature matrix for all digits
        labels: the corresponding ground-truth responses
        '''
        N, D = data.shape
        D_try = int(np.sqrt(D)) # how many features to consider for each split decision

        # initialize the root node
        self.root.data = data
        self.root.labels = labels
        stack = [self.root]

        n_min = 20 # termination criterion: don't split if node contains fewer instances
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min and not node_is_pure(node):
                # Call 'make_decision_split_node()' with 'D_try' randomly selected 
                # feature indices. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                feature_indices=valid_features[np.random.permutation(len(valid_features))[:D_try]]
                #put left and right to end of stack since pop() gets the last item
                stack.extend(make_decision_split_node(node,N,feature_indices)) 
            else:
                # Call 'make_decision_leaf_node()' to turn 'node' into a leaf node.
                node=make_decision_leaf_node(node,N)
                
    def predict(self, x):
        leaf = self.find_leaf(x)
        # compute p(y | x)
        return leaf.response

In [8]:
def score2(S_lambda, S_rho, sorted_labels):
    labels_lambda = sorted_labels[:len(S_lambda)]
    labels_rho = sorted_labels[len(S_lambda):]
    labels = unique(sorted_labels)
    N_lambda_k = []
    N_rho_k =[]  
    for lab in labels:
        N_lambda_k.append(np.count_nonzero(labels_lambda == lab))
        N_rho_k.append(np.count_nonzero(labels_rho == lab))
    Err_lambda = len(S_lambda)*(1 - np.sum(np.square(N_lambda_k/len(S_lambda))))
    Err_rho    = len(S_rho)   *(1 - np.sum(np.square(N_rho_k/len(S_rho))))
    return Err_lambda + Err_rho

def make_decision_split_node(node, feature_indices):
    '''
    node: the node to be split
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape

    # find best feature j (among 'feature_indices') and best threshold t for the split
    best_splits = []
    best_scores = []
    for j in feature_indices:
        sortingIndices=np.argsort(node.data.T[j])
        sorted_data = node.data.T[j][sortingIndices]
        sorted_labels = node.labels[sortingIndices]
        #to do: filter sorted_data,sorted_labels
        x_j = np.insert(sorted_data, 0, 0)
        x_j_plus = np.insert(sorted_data, len(sorted_data), 0)
        t_j = ((x_j + x_j_plus)/2)[1:-2]
        scores = []
        for t in t_j:
            S_lambda = sorted_data[sorted_data <= t]
            x_lambda = S_lambda[[0,-1]]
            S_rho = sorted_data[sorted_data > t]
            x_rho = S_rho[[0,-1]]
            scores.append(score2(S_lambda, S_rho, sorted_labels))
        best_splits.append(t_j[np.argmin(scores)])
        best_scores.append(min(scores))
    
    index=np.argmin(best_scores)
    split_feature = feature_indices[index]
    split_threshold = best_scores[index]
    threshold = best_splits[index]

    
    #just the same code as before:
    # create children, data yet to be added, only definition of boxes
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and bounding boxes
    # according to the optimal split found above
    # split data into left.data and right.data
    subData=node.data[:,split_feature] #vllt index?
    indices=np.argsort(subData)
    splitIndex=np.argwhere(subData[indices]>=threshold)[0][0]
    
    # data smaller than threshold into left 
    left.data=node.data[indices,:][:splitIndex,:]
    right.data =node.data[indices,:][splitIndex:,:]
    
    #split labels
    left.labels=node.labels[indices][:splitIndex]
    right.labels=node.labels[indices][splitIndex:]
    
    # set left.box and right.box:
    m_rho, M_lambda = m.copy(), M.copy()
    m_rho[split_feature], M_lambda[split_feature] = threshold, threshold
    left.box = (m, M_lambda)
    right.box = (m_rho, M)
    
    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left,node.right=left,right
    node.feature=split_feature

    # return the children (to be placed on the stack)
    return left, right  

In [9]:
def make_decision_leaf_node(node):
    '''
    node: the node to become a leaf
    '''
    # compute and store leaf response
    # count number of instances in classes:
    Nlk=np.bincount(node.labels)
    node=Node()
    node.response=Nlk/np.sum(Nlk)
    return node

In [10]:
def node_is_pure(node):
    '''
    check if 'node' ontains only instances of the same digit
    '''
    return True if np.count_nonzero(np.bincount(node.labels)) else False

# Evaluation of Density and Decision Tree

In [11]:
# read and prepare the digits data
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_digits
from sklearn import model_selection
#load digits
digits = load_digits ()
data = digits ["data"]
images = digits ["images"]
target = digits ["target"]
target_names = digits ["target_names"]

In [13]:
# train trees, plot training error confusion matrices, and comment on your results
dens=DensityTree()
dens.train(data[:40],.1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: RuntimeWarning: divide by zero encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:56: RuntimeWarning: invalid value encountered in greater_equal


IndexError: index 0 is out of bounds for axis 0 with size 0

In [42]:
x=[10,2,3,5]
print(x[[0,-1]])

TypeError: list indices must be integers or slices, not list

# Density and Decision Forest

In [7]:
class DensityForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DensityTree() for i in range(n_trees)]
    
    def train(self, data, prior):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

In [35]:
class DecisionForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DecisionTree() for i in range(n_trees)]
    
    def train(self, data, labels):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

# Evaluation of Density and Decision Forest

In [ ]:
# train forests (with 20 trees per forest), plot training error confusion matrices, and comment on your results
... # your code here